## Monarch Initiative
Accelerating precision medicine through Open Data Science

### Download all-associations database
see https://data.monarchinitiative.org/latest/tsv/index.html README file

In [1]:
import requests
from bs4 import BeautifulSoup
import os

# 目标网页地址
base_url = "https://data.monarchinitiative.org/monarch-kg/latest/tsv/all_associations/"
response = requests.get(base_url)

# 检查是否请求成功
if response.status_code != 200:
    print(f"❌ 页面访问失败：{response.status_code}")
else:
    # 解析 HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # 提取所有 href 结尾为 .tsv.gz 的链接
    file_links = [
        a['href'] for a in soup.find_all('a', href=True)
        if a['href'].endswith('.tsv.gz')
    ]

    print(f"🔗 发现 {len(file_links)} 个文件链接")

    # 创建保存目录
    download_dir = "all_associations_files"
    os.makedirs(download_dir, exist_ok=True)

    # 下载所有文件
    for file_name in file_links:
        file_url = file_name if file_name.startswith("http") else base_url + file_name
        local_path = os.path.join(download_dir, os.path.basename(file_name))
        print(f"⬇️ 正在下载: {file_name}")
        try:
            r = requests.get(file_url, stream=True)
            r.raise_for_status()
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        except Exception as e:
            print(f"❌ 下载失败 {file_name}: {e}")



🔗 发现 24 个文件链接
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/latest/tsv/all_associations/association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/latest/tsv/all_associations/causal_gene_to_disease_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/latest/tsv/all_associations/chemical_or_drug_or_treatment_to_disease_or_phenotypic_feature_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/latest/tsv/all_associations/chemical_to_disease_or_phenotypic_feature_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/latest/tsv/all_associations/chemical_to_pathway_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/latest/tsv/all_associations/correlated_gene_to_disease_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/latest/tsv/all_associations/disease_or_phenotypic_feature_to_genetic_inheritance_association.all.tsv.gz
⬇️ 正在下载: https:/

In [2]:
import gzip
import shutil
import os

# 定义输入目录和输出目录
input_dir = "all_associations_files"
output_dir = "all_associations_unzipped"
os.makedirs(output_dir, exist_ok=True)

# 遍历 input_dir 中所有 .gz 文件
for filename in os.listdir(input_dir):
    if filename.endswith(".tsv.gz"):
        input_path = os.path.join(input_dir, filename)
        output_filename = filename.replace(".gz", "")  # 移除.gz后缀
        output_path = os.path.join(output_dir, output_filename)

        print(f"🧩 解压中: {filename} -> {output_filename}")
        try:
            with gzip.open(input_path, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        except Exception as e:
            print(f"❌ 解压失败 {filename}: {e}")

print("✅ 全部解压完成！")


🧩 解压中: causal_gene_to_disease_association.all.tsv.gz -> causal_gene_to_disease_association.all.tsv
🧩 解压中: disease_or_phenotypic_feature_to_genetic_inheritance_association.all.tsv.gz -> disease_or_phenotypic_feature_to_genetic_inheritance_association.all.tsv
🧩 解压中: genotype_to_disease_association.all.tsv.gz -> genotype_to_disease_association.all.tsv
🧩 解压中: chemical_or_drug_or_treatment_to_disease_or_phenotypic_feature_association.all.tsv.gz -> chemical_or_drug_or_treatment_to_disease_or_phenotypic_feature_association.all.tsv
🧩 解压中: association.all.tsv.gz -> association.all.tsv
🧩 解压中: gene_to_pathway_association.all.tsv.gz -> gene_to_pathway_association.all.tsv
🧩 解压中: macromolecular_machine_to_molecular_activity_association.all.tsv.gz -> macromolecular_machine_to_molecular_activity_association.all.tsv
🧩 解压中: genotype_to_gene_association.all.tsv.gz -> genotype_to_gene_association.all.tsv
🧩 解压中: correlated_gene_to_disease_association.all.tsv.gz -> correlated_gene_to_disease_association.all

### Merge all asscociations data

In [3]:
import pandas as pd
import os

# Directory containing the unzipped files
input_dir = "all_associations_unzipped"

# Output file name for the merged result
output_file = "merged_associations_all.tsv"

# List to store all DataFrames
df_list = []

# Iterate through all .tsv files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".tsv"):
        file_path = os.path.join(input_dir, filename)
        print(f"📥 Reading: {filename}")
        try:
            # Read each .tsv file into a DataFrame
            df = pd.read_csv(file_path, sep='\t', low_memory=False)
            df_list.append(df)
        except Exception as e:
            print(f"❌ Failed to read {filename}: {e}")

# Concatenate all DataFrames into one
if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    # Save the merged DataFrame as a new TSV file
    merged_df.to_csv(output_file, sep='\t', index=False)
    print(f"✅ All files have been merged and saved to: {output_file}")
else:
    print("⚠️ No files found to merge.")

📥 Reading: gene_to_expression_site_association.all.tsv
📥 Reading: macromolecular_machine_to_biological_process_association.all.tsv
📥 Reading: chemical_to_pathway_association.all.tsv
📥 Reading: correlated_gene_to_disease_association.all.tsv
📥 Reading: variant_to_phenotypic_feature_association.all.tsv
📥 Reading: genotype_to_phenotypic_feature_association.all.tsv
📥 Reading: variant_to_disease_association.all.tsv
📥 Reading: disease_or_phenotypic_feature_to_genetic_inheritance_association.all.tsv
📥 Reading: causal_gene_to_disease_association.all.tsv
📥 Reading: genotype_to_variant_association.all.tsv
📥 Reading: chemical_or_drug_or_treatment_to_disease_or_phenotypic_feature_association.all.tsv
📥 Reading: disease_or_phenotypic_feature_to_location_association.all.tsv
📥 Reading: gene_to_gene_homology_association.all.tsv
📥 Reading: pairwise_gene_to_gene_interaction.all.tsv
📥 Reading: disease_to_phenotypic_feature_association.all.tsv
📥 Reading: chemical_to_disease_or_phenotypic_feature_association

### Reformat the data files as the BioMedGPS format

In [4]:
import pandas as pd

df = pd.read_csv("merged_associations_all.tsv", sep="\t", low_memory=False)
df.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN


In [5]:
# Check the source database and ID prefix for each type of subject entity
# Loop through each unique subject_category
for stype in df["subject_category"].unique():
    # Filter all rows with the current subject_category
    sub_df = df[df["subject_category"] == stype]
    
    # Extract the prefix before the colon in the subject ID (e.g., "MGI:12345" -> "MGI")
    prefixes = sub_df["subject"].str.split(":").str[0].unique()
    
    # Print the result
    print(f"{stype}: {prefixes}")

biolink:Gene: ['FB' 'NCBIGene' 'HGNC' 'MGI' 'WB' 'RGD' 'ZFIN' 'SGD' 'PomBase'
 'dictyBase' 'Xenbase']
biolink:SequenceVariant: ['RGD' 'MGI' 'CLINVAR' 'CAID' 'ZFIN']
biolink:Protein: ['PR']
biolink:ChemicalEntity: ['CHEBI']
biolink:Genotype: ['ZFIN' 'MGI' 'RGD']
biolink:Disease: ['MONDO' 'MPATH']
biolink:NamedThing: ['MAXO' 'DDANAT' 'BFO' 'BSPO' 'FBcv' 'FBdv' 'OBA' 'PATO' 'PHIPO' 'RO'
 'WBLS' 'MGPO']
biolink:PhenotypicFeature: ['XPO' 'ZP' 'DDPHENO' 'FYPO' 'UPHENO' 'WBPhenotype' 'HP' 'MP' 'NBO']
biolink:AnatomicalEntity: ['ZFA' 'EMAPA' 'FBbt' 'UBERON' 'WBbt' 'XAO']
biolink:LifeStage: ['ZFS' 'HSAPDV']
biolink:MolecularEntity: ['CHR' 'SO']
biolink:Cell: ['CL']
biolink:BiologicalProcess: ['GO']
biolink:MolecularActivity: ['GO']
biolink:CellularComponent: ['GO']
biolink:OrganismTaxon: ['NCBITaxon']


In [6]:
# Loop through each unique object_category
for stype in df["object_category"].unique():
    # Filter all rows with the current object_category
    object_df = df[df["object_category"] == stype]
    
    # Extract the prefix before the colon in the object ID（e.g. MGI:12345 -> MGI）
    object_prefixes = object_df["object"].str.split(":").str[0].unique()
    
    # Print the re
    print(f"{stype}: {object_prefixes}")

biolink:AnatomicalEntity: ['FBbt' 'UBERON' 'EMAPA' 'WBbt' 'ZFA' 'XAO']
biolink:Cell: ['CL']
biolink:CellularComponent: ['GO']
biolink:BiologicalProcess: ['GO']
biolink:Pathway: ['Reactome']
biolink:Disease: ['MONDO' 'MPATH']
biolink:PhenotypicFeature: ['MP' 'HP' 'ZP' 'FYPO' 'XPO' 'DDPHENO' 'WBPhenotype' 'UPHENO' 'NBO']
biolink:SequenceVariant: ['MGI' 'RGD' 'ZFIN']
biolink:Gene: ['RGD' 'NCBIGene' 'Xenbase' 'ZFIN' 'FB' 'MGI' 'WB' 'SGD' 'PomBase'
 'dictyBase' 'HGNC']
biolink:MolecularActivity: ['GO']
biolink:NamedThing: ['GO' 'BSPO' 'BFO' 'PATO' 'CLM' 'DDANAT' 'FBcv' 'FBdv' 'OBA' 'WBLS' 'MAXO']
biolink:LifeStage: ['ZFS' 'HSAPDV']
biolink:ChemicalEntity: ['CHEBI']
biolink:OrganismTaxon: ['NCBITaxon']
biolink:MolecularEntity: ['CHR' 'SO']
biolink:Protein: ['PR']


In [7]:
df_mapping = df
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN


#### MGI → Entrez ID 映射

In [8]:
# 读取 Excel 或 TSV 映射文件（请根据实际文件路径和格式选择）
mouse_mapping_df = pd.read_csv("/Users/zhuzhixing/KG/biomedgps-data/graph_data/mapping/human_mouse_gene_mappings.tsv", sep="\t") # 或 pd.read_csv("your_mapping.tsv", sep="\t")

# 构造映射字典（确保列名对应）
mgi_to_entrez = dict(zip(mouse_mapping_df["mgi_id_mouse"], mouse_mapping_df["entrez_id_mouse"]))

# 初始化 subject_mapping 列为原始 subject
df_mapping["subject_mapping"] = df_mapping["subject"]

# 替换 MGI 开头的行
df_mapping["subject_mapping"] = df_mapping["subject_mapping"].apply(
    lambda x: mgi_to_entrez.get(x, x) if isinstance(x, str) and x.startswith("MGI:") else x
)

In [9]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,subject_mapping
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526


#### CHEBI ID → DrugBank ID 映射（基于 xrefs）

In [11]:
import re

# 1. 读取原始实体文件（请根据实际路径和分隔符修改）
entities_df = pd.read_csv("/Users/zhuzhixing/KG/biomedgps-data/graph_data/entities.tsv", sep="\t", low_memory=False)  # 或 pd.read_excel("entities.xlsx")

# 2. 提取 CHEBI ID（支持 xrefs 中包含多个以 | 分隔的标识）
def extract_chebi(xrefs):
    if pd.isna(xrefs):
        return None
    matches = re.findall(r"CHEBI:\d+", xrefs)
    return matches[0] if matches else None

entities_df["chebi_id_extracted"] = entities_df["xrefs"].apply(extract_chebi)

# 3. 保留 chebi_id_extracted 非空的行
chebi_only_df = entities_df[entities_df["chebi_id_extracted"].notna()]

# 4. 构建 CHEBI → DrugBank 映射字典
chebi_to_drugbank = dict(zip(chebi_only_df["chebi_id_extracted"], chebi_only_df["id"]))

# 5. 替换 chebi_input_df["id"] 中的 CHEBI ID 为 DrugBank ID（如果在映射表中）
df_mapping["subject_mapping"] = df_mapping["subject_mapping"].apply(lambda x: chebi_to_drugbank.get(x, x))


In [12]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,subject_mapping
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526


#### subject_label → DrugBank（基于 name_clean 映射）

In [13]:
# 筛选 CHEBI 类型的 compound 实体
compound_df = entities_df[entities_df["label"] == "Compound"].copy()

# 使用 compound_df，处理名称为映射键
compound_df["name_clean"] = compound_df["name"].str.strip().str.lower()

# 构建 name → DrugBank ID 映射字典
name_to_drugbank = dict(zip(compound_df["name_clean"], compound_df["id"]))

# 初始化 subject_label_mapping 列
df_mapping["subject_label_mapping"] = None

# 对 ChemicalEntity 类型的数据进行 name 匹配映射
mask_chemical = df["subject_category"] == "biolink:ChemicalEntity"
df_mapping.loc[mask_chemical, "subject_label_clean"] = df_mapping.loc[mask_chemical, "subject_label"].str.strip().str.lower()
df_mapping.loc[mask_chemical, "subject_label_mapping"] = df_mapping.loc[mask_chemical, "subject_label_clean"].map(name_to_drugbank)



In [14]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,subject_mapping,subject_label_mapping,subject_label_clean
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN


#### 根据 label 映射修正 subject_mapping

In [15]:
# 只处理 subject_label_mapping 有值的行
mask_non_null = df_mapping["subject_label_mapping"].notna()

# 如果 mapping 不一致，用 label_mapping 替换
df_mapping.loc[mask_non_null & (df_mapping["subject_mapping"] != df_mapping["subject_label_mapping"]),
       "subject_mapping"] = df_mapping.loc[mask_non_null & (df_mapping["subject_mapping"] != df_mapping["subject_label_mapping"]),
                                   "subject_label_mapping"]


In [16]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,subject_mapping,subject_label_mapping,subject_label_clean
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,None,NaN


#### 对object做上述同样的映射

In [17]:
#初始化 object_mapping 列
df_mapping["object_mapping"] = df_mapping["object"]
#根据上文构建的 CHEBI → DrugBank 字典（基于 xrefs），应用映射到 object_mapping 列
df_mapping["object_mapping"] = df_mapping["object_mapping"].apply(
    lambda x: chebi_to_drugbank.get(x, x) if isinstance(x, str) and x.startswith("CHEBI:") else x
)
#基于 object_label 做进一步名称映射
# 初始化映射列
df_mapping["object_label_mapping"] = None

# 筛选 ChemicalEntity 行
mask_chem_obj = df_mapping["object_category"] == "biolink:ChemicalEntity"

# 标准化名称
df_mapping.loc[mask_chem_obj, "object_label_clean"] = df_mapping.loc[mask_chem_obj, "object_label"].str.strip().str.lower()

# 应用名称映射
df_mapping.loc[mask_chem_obj, "object_label_mapping"] = df_mapping.loc[mask_chem_obj, "object_label_clean"].map(name_to_drugbank)

# 替换 object_mapping（优先使用 label 映射值）
mask_non_null_obj = df_mapping["object_label_mapping"].notna()
df_mapping.loc[
    mask_non_null_obj & (df_mapping["object_mapping"] != df_mapping["object_label_mapping"]),
    "object_mapping"
] = df_mapping.loc[
    mask_non_null_obj & (df_mapping["object_mapping"] != df_mapping["object_label_mapping"]),
    "object_label_mapping"
]


In [18]:
#其他的ID映射
# 替换 NCBIGene: → ENTREZ:，适用于 subject_mapping 和 object_mapping
df_mapping["subject_mapping"] = df_mapping["subject_mapping"].str.replace(r"^NCBIGene:", "ENTREZ:", regex=True)
df_mapping["object_mapping"] = df_mapping["object_mapping"].str.replace(r"^NCBIGene:", "ENTREZ:", regex=True)

# 替换 Reactome → REACT，适用于 subject_mapping 和 object_mapping
df_mapping["subject_mapping"] = df_mapping["subject_mapping"].str.replace(r"^Reactome:", "REACT:", regex=True)
df_mapping["object_mapping"] = df_mapping["object_mapping"].str.replace(r"^Reactome:", "REACT:", regex=True)

In [19]:
#删除多余的映射列
columns_to_drop = [
    "subject_label_clean",
    "subject_label_mapping",
    "object_label_clean",
    "object_label_mapping"
]
df_mapping.drop(columns=columns_to_drop, inplace=True, errors="ignore")
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,subject_mapping,object_mapping
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00001102
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,MMO:0000658,PMID:24204300,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00001060
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,MMO:0000658,FB:FBrf0219073,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001


#### 对subject_category和object_category进行格式化处理

In [20]:
# 拆分 biolink:前缀，生成新的列
df_mapping["source_type"] = df_mapping["subject_category"].str.replace("biolink:", "", regex=False)
df_mapping["target_type"] = df_mapping["object_category"].str.replace("biolink:", "", regex=False)

In [21]:
#替换 AnatomicalEntity → Anatomy, 适用于source_type和target_type
df_mapping["source_type"] = df_mapping["source_type"].replace("AnatomicalEntity", "Anatomy")
df_mapping["target_type"] = df_mapping["target_type"].replace("AnatomicalEntity", "Anatomy")

#替换 PhenotypicFeature → Phenotype, 适用于source_type和target_type
df_mapping["source_type"] = df_mapping["source_type"].replace("PhenotypicFeature", "Phenotype")
df_mapping["target_type"] = df_mapping["target_type"].replace("PhenotypicFeature", "Phenotype")

#替换 MolecularActivity → MolecularFunction, 适用于source_type和target_type
df_mapping["source_type"] = df_mapping["source_type"].replace("MolecularActivity", "MolecularFunction")
df_mapping["target_type"] = df_mapping["target_type"].replace("MolecularActivity", "MolecularFunction")

#替换 ChemicalEntity → Compound, 适用于source_type和target_type
df_mapping["source_type"] = df_mapping["source_type"].replace("ChemicalEntity", "Compound")
df_mapping["target_type"] = df_mapping["target_type"].replace("ChemicalEntity", "Compound")


# 创建新列 relation_type = predicate:source_type:target_type
df_mapping["relation_type"] = df_mapping["predicate"] + ":" + df_mapping["source_type"] + ":" + df_mapping["target_type"]

# 替换 biolink:predicate:source:target 为 biolink::predicate::source:target
df_mapping["relation_type"] = df_mapping["relation_type"].str.replace(
    r"^biolink:([^:]+):", r"biolink::\1::", regex=True
)

In [22]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,...,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,subject_mapping,object_mapping,source_type,target_type,relation_type
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00001102,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00001060,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy


#### 对relation_type进行标准化处理

In [23]:
df_mapping["relation_type"].unique()

array(['biolink::expressed_in::Gene:Anatomy',
       'biolink::expressed_in::Gene:Cell',
       'biolink::expressed_in::Gene:CellularComponent',
       'biolink::actively_involved_in::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_or_within::Gene:BiologicalProcess',
       'biolink::acts_upstream_of::Gene:BiologicalProcess',
       'biolink::actively_involved_in::SequenceVariant:BiologicalProcess',
       'biolink::acts_upstream_of_positive_effect::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_negative_effect::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_or_within_positive_effect::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_or_within_negative_effect::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_or_within::Protein:BiologicalProcess',
       'biolink::acts_upstream_of::Protein:BiologicalProcess',
       'biolink::actively_involved_in::Protein:BiologicalProcess',
       'biolink::acts_upstream_of_or_within_negative_effect

In [24]:
df_mapping["relation_type"].nunique()

189

In [25]:
relation_counts_df = df_mapping["relation_type"].value_counts().reset_index()
relation_counts_df.columns = ["relation_type", "count"]
relation_counts_df.to_csv("relation_type_counts.csv", index=False)


In [26]:
import pandas as pd

# 1. 加载 Excel 文件中 sheet='relation_type' 的合法关系类型
relation_file_path = "/Users/zhuzhixing/KG/relation_types_20250524.xlsx"  # 替换为你的路径
relation_types_df = pd.read_excel(relation_file_path, sheet_name="relation_type")

# 2. 获取所有合法的 relation_type（去除缺失值）
valid_relations = relation_types_df["relation_type"].dropna().unique()

# 3. 筛选 df_mapping 中 relation_type 在合法列表中的行
filtered_df = df_mapping[df_mapping["relation_type"].isin(valid_relations)]

filtered_df.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,...,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,subject_mapping,object_mapping,source_type,target_type,relation_type
0,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
1,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
2,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001102,larval ventral nerve cord,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00001102,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
3,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00001060,embryonic brain,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00001060,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
4,FB:FBgn0053526,PNUTS,biolink:Gene,NCBITaxon:7227,Drosophila melanogaster,NaN,biolink:expressed_in,FBbt:00000001,organism,biolink:AnatomicalEntity,...,NaN,infores:flybase,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,FB:FBgn0053526,FBbt:00000001,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy


In [27]:
len(filtered_df)

13135236

In [28]:
len(df_mapping)

14634548

In [29]:
formatted_df = pd.DataFrame()
formatted_df["source_name"] = df_mapping["subject_label"]
formatted_df["source_type"] = df_mapping["source_type"]
formatted_df["source_id"] = df_mapping["subject"]
formatted_df["target_name"] = df_mapping["object_label"]
formatted_df["target_type"] = df_mapping["target_type"]
formatted_df["target_id"] = df_mapping["object"]
formatted_df["relation_type"] = df_mapping["relation_type"]

formatted_df["resource"] = "biolink"

invalid_formatted_df = formatted_df[formatted_df["source_id"].isna() | formatted_df["target_id"].isna()]
invalid_formatted_df.to_csv("invalid_formatted_biolink.tsv", index=False, sep="\t")

formatted_df = formatted_df[~formatted_df["source_id"].isna() & ~formatted_df["target_id"].isna()]
formatted_df.to_csv("formatted_biolink.tsv", index=False, sep="\t")


In [30]:
formatted_df.head()

,source_name,source_type,source_id,target_name,target_type,target_id,relation_type,resource
0,PNUTS,Gene,FB:FBgn0053526,organism,Anatomy,FBbt:00000001,biolink::expressed_in::Gene:Anatomy,biolink
1,PNUTS,Gene,FB:FBgn0053526,organism,Anatomy,FBbt:00000001,biolink::expressed_in::Gene:Anatomy,biolink
2,PNUTS,Gene,FB:FBgn0053526,larval ventral nerve cord,Anatomy,FBbt:00001102,biolink::expressed_in::Gene:Anatomy,biolink
3,PNUTS,Gene,FB:FBgn0053526,embryonic brain,Anatomy,FBbt:00001060,biolink::expressed_in::Gene:Anatomy,biolink
4,PNUTS,Gene,FB:FBgn0053526,organism,Anatomy,FBbt:00000001,biolink::expressed_in::Gene:Anatomy,biolink


In [31]:
len(formatted_df)

14634548

In [32]:
import os
import os.path as osp
import subprocess


def format_biolink(filename):
    def get_project_root():
        try:
            return osp.dirname(osp.dirname(os.getcwd()))
        except Exception as e:
            raise RuntimeError(f"Failed to determine project root: {e}")

    try:
        root_dir = get_project_root()
        print(f"Project root directory: {root_dir}")
    except RuntimeError as e:
        print(e)
        exit(1)

    database = "customdb"
    relations_path = osp.join(
        root_dir,
        "relations",
        "Monarch",
        filename,
    )
    output_dir = osp.join(
        root_dir, "formatted_relations", "monarch"
    )
    entities_path = osp.join(root_dir, "entities.tsv")
    log_file = osp.join(output_dir, "log.txt")
    relation_types_file = osp.join(root_dir, "relation_types.tsv")

    command = [
        "graph-builder",
        "--database",
        database,
        "-d",
        relations_path,
        "-o",
        output_dir,
        "-f",
        entities_path,
        "-n",
        "20",
        "--download",
        "--skip",
        "-l",
        log_file,
        "--debug",
        "--relation-type-dict-fpath",
        relation_types_file,
    ]

    print("Executing command:", " ".join(command))

    try:
        subprocess.run(command, check=True)
    except FileNotFoundError:
        print(
            "Error: 'graph-builder' command not found. Make sure it is installed and available in the PATH."
        )
        exit(1)
    except subprocess.CalledProcessError as e:
        print(f"Error: Command execution failed with return code {e.returncode}")
        print(f"Output: {e.output}")
        exit(1)
    except Exception as e:
        print(f"Unexpected error: {e}")
        exit(1)

In [ ]:
format_biolink("formatted_biolink.tsv")

Project root directory: /Users/zhuzhixing/KG/biomedgps-data/graph_data
Executing command: graph-builder --database customdb -d /Users/zhuzhixing/KG/biomedgps-data/graph_data/relations/Monarch/formatted_biolink.tsv -o /Users/zhuzhixing/KG/biomedgps-data/graph_data/formatted_relations/monarch -f /Users/zhuzhixing/KG/biomedgps-data/graph_data/entities.tsv -n 20 --download --skip -l /Users/zhuzhixing/KG/biomedgps-data/graph_data/formatted_relations/monarch/log.txt --debug --relation-type-dict-fpath /Users/zhuzhixing/KG/biomedgps-data/graph_data/relation_types.tsv


2025-07-25 20:57:42 - cli:171 - INFO - Run jobs with (output_dir: /Users/zhuzhixing/KG/biomedgps-data/graph_data/formatted_relations/monarch, db file/directory: /Users/zhuzhixing/KG/biomedgps-data/graph_data/relations/Monarch/formatted_biolink.tsv, databases: ('customdb',), download: True, skip: True)
2025-07-25 20:57:45 - base_parser:229 - INFO - Using allow_ignore_checking_errors=all to ignore the checking errors.
